In [1]:
import nltk
from nltk.tokenize import WhitespaceTokenizer
from string import punctuation
import re
import string
import pandas as pd
import random

import logging
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
import pymorphy2
import gensim
from sklearn.preprocessing import MinMaxScaler

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

E:\programs\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def take_text(fin):
    f = open(fin, 'r', encoding = 'utf-8')
    twi = []
    cont = []
    l = f.readlines()
    for i in l:
        i = i.replace('"\n', '')
        if i[0] == '"':
            lst = i[1:].split('";"')
        else:
            lst = i.split('";"')
        if len(lst) == 12:
            twi.append(lst)
        elif cont == []:
            cont += lst
        elif '\n' in cont[-1] and lst != []:
            c = cont[-1]
            cont = cont[:-1]
            cont.append(c+lst[0])
            cont += lst[1:]
            if len(cont) == 12:
                twi.append(cont)
                cont = []
    f.close()
    
    only_twits = []
    for line in twi:
        only_twits.append(line[3])
    
    return only_twits

In [3]:
def spacing(text):
    spaces = re.findall('[^\\w]((?:\\w ){3,}\\w)[ '+string.punctuation+'—–…“”«»'+']',text)
    for elem in spaces:
        text = text.replace(elem,elem.replace(' ',''))
    return text

In [4]:
def normalize(l):
    l = re.sub(r"((https?:\/\/|www)|\w+\.(\w{2-3}))([\w\!#$&-;=\?\-\[\]~]|%[0-9a-fA-F]{2})+", '', l) #убираем ссылки
    l = re.sub(r"(?:@\w+)", '', l)                                        # убираем пользователя
    l = re.sub(r"[\w.+-]+@[\w-]+\.(?:[\w-]\.?)+[\w-]", '', l)             # убираем email
    l = re.sub(r"(?:I'm at .*? in .*?(?: w/ @[\w])?|\(@ .*?\))", '', l)   # убираем геолокации
    l = l.replace('RT', '')                                              # убираем RT
    exclude = string.punctuation + '0123456789' + u'—' + u'«»'
    regex = re.compile('[%s]' % re.escape(exclude))
    l = regex.sub(' ', l)
    lower_up = [m.start() for m in re.finditer(r"[a-zа-яё]{1}[A-ZА-ЯЁ]{1}", l)]   # добавляем пробел между lower- и uppercase HiWorld => Hi World
    for i in reversed(lower_up):
        l = l[:i+1] + ' ' + l[i+1:]
    l = re.sub(u'(\w)\\1{2,}', u'\\1\\1', l)   # буквы больше двух раз подряд
    l = spacing(l)                                # р а з р я д к а
    l = l.lower()                                                         # уменьшаем регистр у всех букв
    return l

In [5]:
def norm_tw(twits):
    norm_twits = []
    for i in range(len(twits)):
        norm_twits.append(normalize(twits[i]))
    return norm_twits

In [6]:
twits = take_text('positive.csv')
neg_twits = take_text('negative.csv')

In [7]:
twits = random.sample(twits, len(twits)//10)
neg_twits = random.sample(neg_twits, len(neg_twits)//10)

In [8]:
norm_twits = norm_tw(twits)
neg_norm_twits = norm_tw(neg_twits)

In [9]:
data1 = pd.DataFrame.from_dict({'text':norm_twits})
data1['class'] = 1
data2 = pd.DataFrame.from_dict({'text':neg_norm_twits})
data2['class'] = 0

In [10]:
data = pd.concat([data1, data2])
data = data.sample(frac=1).reset_index(drop=True)

In [11]:
data.head()

,text,class
0,ты начинаешь злиться а я улыбаюсь я счастлива ...,1
1,но димон же зол на него поэтому стоит увиде...,1
2,вроде на улице зима а до меня это еще не дохо...,1
3,сел короче проверить мозги после нг пробный е...,0
4,место того чтоб учить я слушаю музыку класс,0


In [12]:
# save as csv

data.to_csv('dataframe.csv', sep='\t', encoding='utf-8')